# Forward-backward splitting for time-varying graphical lasso
This notebook shows how to minimise the time-varying graphical lasso with element-wise penalty norms across time-points.

First of all, as always, let's create a bunch of data.
For this task, we generate eah variable to change according to a certain behaviour which can be described as evolution via tigonometric functions, such as `sin` and `cos`.

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from scipy.spatial.distance import squareform
from regain import datasets, utils

from sklearn.datasets import load_iris
from sklearn.svm import SVC 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, ShuffleSplit

from skopt.searchcv import BayesSearchCV
from skopt.space import Real, Categorical, Integer

In [2]:
# np.random.seed(7)

# fs = 10e3
# N = 100
# amp = 2*np.sqrt(2)
# freq = 1.0
# noise_power = 0.001 * fs / 2
# time = np.arange(N) / fs
# z = amp*np.sin(2*np.pi*freq*time)
# z += np.random.normal(scale=np.sqrt(noise_power), size=time.shape)
# plt.plot(z);

In [3]:
# T = 4

# x = np.tile(np.linspace(0, T-1, T), (n_interactions, 1))
# zz = amp * signal.square(2 * np.pi * freq * x + phase, duty=.5)
# plt.plot(x.T, zz.T);

Generate the data starting from the inverse covariance matrices.

In [4]:
np.random.seed(7)

# square
n_samples = 100
n_dim_obs = 20
T = 10

data = {}
data['square'] = datasets.make_dataset(n_samples=n_samples, n_dim_obs=n_dim_obs, n_dim_lat=0, T=T,
                             time_on_axis='last',
                             mode='sin', shape='square', closeness=2.4, normalize=1)

# smooth
n_samples = 100
n_dim_obs = 20
T = 10

data['smooth'] = datasets.make_dataset(n_samples=n_samples, n_dim_obs=n_dim_obs, n_dim_lat=0, T=T,
                             time_on_axis='last',
                             mode='sin', shape='smooth', closeness=2.4, normalize=1)

# plt.step(np.array([squareform(y, checks=None) for y in data.thetas]), '-|');
# plt.savefig("/home/fede/Dropbox/Latent variables networks/forward backward time varying graphical lasso/smooth_signal.pdf")

### Let's run 

In [5]:
# X = data.data
# X_tr, X_ts = train_test_split(X)

In [449]:
import pandas as pd
from functools import partial
from regain import update_rules; reload(update_rules);
from regain.forward_backward import time_graph_lasso_; reload(time_graph_lasso_)

from sklearn.covariance import GraphLasso, GraphLassoCV
import sys; sys.path.append("/home/fede/src/TVGL")
import inferGraphL1; reload(inferGraphL1)
import TVGL; reload(TVGL)

# use:
# beta = 2.1, norm = 1
# beta = 5.05, norm = 2
# prepare dataframe for results

methods = ['TGL-FB ($\ell_{21}$)', 'TGL-FB ($\ell_1$)', 'GL', 'TVGL ($\ell_{21}$)', 'TVGL ($\ell_1$)']

scores = sorted(['iter', 'accuracy',  'average_precision',  'balanced_accuracy',  'dor',  'f1',  'fall_out',  'false_omission_rate',  'fdr',  'fn',  'fp',  'miss_rate',  'nlr',  'npv',  'plr',  'precision',  'prevalence',  'recall',  'specificity',  'tn',  'tp'])
evolution = sorted(['square', 'smooth'])

rows = methods
cols = pd.MultiIndex.from_product([evolution, scores], names=('evolution', 'score'))
# rows = pd.MultiIndex.from_product([methods, n_times], names=('method','time'))

dff = pd.DataFrame(columns=cols, index=rows)
idx = pd.IndexSlice

In [450]:
def run_performance(beta=1, evolution='square'):
    X = data[evolution].data
    
    error_function = partial(utils.structure_error, data[evolution].thetas,
                             no_diagonal=0, thresholding=0, eps=1e-3)
    tglfb = time_graph_lasso_.TimeGraphLassoForwardBackward(
        verbose=0, gamma=1, alpha='max', beta=beta,
        delta=.0001, choose='lamda',
        lamda=1, tol=1e-5, eps=0.7,
        time_norm=2, max_iter=300, time_on_axis='last').fit(X)
#     print(tglfb.alpha_max(X))
#     return
    res = error_function(tglfb.precision_)
    res['iter'] = tglfb.n_iter_
    dff.loc['TGL-FB ($\ell_{21}$)', idx[evolution, :]] = [res[x] for x in scores]
    
    tglfb = tglfb.set_params(time_norm=1).fit(X)
    res = error_function(tglfb.precision_)
    res['iter'] = tglfb.n_iter_
    dff.loc['TGL-FB ($\ell_1$)', idx[evolution, :]] = [res[x] for x in scores]

    gl = GraphLasso(alpha=tglfb.alpha)
#     gl = GraphLassoCV(alphas=100)
    precision_gl = np.array([gl.fit(x).precision_.copy() for x in X.transpose(2,0,1)])
    res = error_function(precision_gl)
    res['iter'] = gl.n_iter_
    dff.loc['GL', idx[evolution, :]] = [res[x] for x in scores]
    
    thetaSet, empCovSet, status, gvx = TVGL.TVGL(
        np.vstack(X.transpose(2,0,1)), X.shape[0],
        lamb=tglfb.alpha, beta=tglfb.beta, indexOfPenalty=2, verbose=False)

    res = error_function(np.array(thetaSet))
    res['iter'] = gvx.n_iter_
    dff.loc['TVGL ($\ell_{21}$)', idx[evolution, :]] = [res[x] for x in scores]
    
    thetaSet, empCovSet, status, gvx = TVGL.TVGL(
        np.vstack(X.transpose(2,0,1)), X.shape[0],
        lamb=tglfb.alpha, beta=tglfb.beta, indexOfPenalty=1, verbose=False)
    res = error_function(np.array(thetaSet))
    res['iter'] = gvx.n_iter_
    dff.loc['TVGL ($\ell_1$)', idx[evolution, :]] = [res[x] for x in scores]

In [453]:
run_performance(beta=bscv.best_params_['beta'], evolution='square')
run_performance(beta=bscv_smooth.best_params_['beta'], evolution='smooth')
# run_performance(beta=2.1, evolution='smooth')

/home/fede/src/slipguru/regain/regain/forward_backward/time_graph_lasso_.py:226: UserWarning: precision is not positive definite.
  objective, n_samples=n_samples, emp_cov=emp_cov,


Use l-2 penalty function
10
lambda = 0.1, beta = 1
Use l-1 penalty function
10
lambda = 0.1, beta = 1


In [454]:
# dff['smooth'].sort_values("f1", ascending=False)['iter']
dff['square'].sort_values("f1", ascending=False)

score,accuracy,average_precision,balanced_accuracy,dor,f1,fall_out,false_omission_rate,fdr,fn,fp,...,miss_rate,nlr,npv,plr,precision,prevalence,recall,specificity,tn,tp
TGL-FB ($\ell_{21}$),0.5895,0.817554,0.5,0,0.741743,1,0,0.4105,0,1642,...,0,0,1,1,0.5895,0.5895,1,0,0,2358
TGL-FB ($\ell_1$),0.5895,0.744296,0.5,0,0.741743,1,0,0.4105,0,1642,...,0,0,1,1,0.5895,0.5895,1,0,0,2358
GL,0.5645,0.725211,0.579025,1.92624,0.574083,0.339829,0.522046,0.322171,1184,558,...,0.50212,0.760592,0.477954,1.46509,0.677829,0.5895,0.49788,0.660171,1084,1174
TVGL ($\ell_1$),0.542,0.64296,0.600625,4.85264,0.412821,0.0718636,0.529339,0.154856,1714,118,...,0.726887,0.783168,0.470661,3.80043,0.845144,0.5895,0.273113,0.928136,1524,644
TVGL ($\ell_{21}$),0.467,0.573101,0.547922,0,0.174923,0,0.564918,0,2132,0,...,0.904156,0.904156,0.435082,0,1,0.5895,0.0958439,1,1642,226


In [439]:
evolution = 'square'
X = data[evolution].data
reload(time_graph_lasso_)
tglfb = time_graph_lasso_.TimeGraphLassoForwardBackward(
    verbose=0, alpha='max', beta=2,
    delta=1e-5, eps=0.7, time_norm=1, max_iter=300, time_on_axis='last',
    choose='gamma', tol=1e-5, lamda_criterion='c'
).fit(X)

In [440]:
tglfb.n_iter_

300

In [442]:
error_function = partial(utils.structure_error, data[evolution].thetas,
                             no_diagonal=0, thresholding=1, eps=1e-4)

res = error_function(tglfb.precision_)
res

{'accuracy': 0.59,
 'average_precision': 0.7447260620913716,
 'balanced_accuracy': 0.5006090133982948,
 'dor': 0,
 'f1': 0.7419760855884204,
 'fall_out': 0.9987819732034104,
 'false_omission_rate': 0.0,
 'fdr': 0.4102051025512756,
 'fn': 0,
 'fp': 1640,
 'miss_rate': 0.0,
 'nlr': 0.0,
 'npv': 1.0,
 'plr': 1.001219512195122,
 'precision': 0.5897948974487244,
 'prevalence': 0.5895,
 'recall': 1.0,
 'specificity': 0.001218026796589525,
 'tn': 2,
 'tp': 2358}

### BayesOptimisation
Since we have lots of hyper-parameters, we rely on a Bayesian optimisation procedure in order to select the best hyper-parameters, treating the scoring function of our algorithm as a black-box for the gaussian process underlying the Bayesian optimisation.

Such procedure is performed via the `scikit-optimize` package.

In [ ]:
from regain import utils; reload(utils)
from regain import prox; reload(prox);
from regain.forward_backward import time_graph_lasso_; reload(time_graph_lasso_)

from skopt import searchcv; reload(searchcv)

X = data['square'].data

domain = {#'alpha': Real(1e-1, 3, prior='uniform'),
          'beta': Real(1e-1, 1e1, prior='uniform'),
#           'time_norm': Categorical([1, 2])
#           'eps': Categorical([0.5, 0.7, 0.8, 0.9])
         }

mdl = time_graph_lasso_.TimeGraphLassoForwardBackward(
    verbose=0, tol=1e-5, max_iter=200, gamma=1, alpha='max', beta=2.1, time_norm=1,
    time_on_axis='last', eps=0.7, delta=1e-5, choose='lamda', lamda_criterion='c')
    
cv = ShuffleSplit(5, test_size=0.2)
    
bscv = searchcv.BayesSearchCV(
    mdl, domain, n_iter=200, cv=cv, verbose=0, n_jobs=1, iid=True, n_points=3,
    error_score=-3e5)

def on_step(optim_result):
    score = bscv.best_score_
#     print("best score: %s" % score)

bscv.fit(X, callback=on_step)
# mdl.fit(data_grid)

/home/fede/src/slipguru/regain/regain/forward_backward/time_graph_lasso_.py:299: UserWarning: Objective did not converge.
  if iteration_ == 0:


In [365]:
bscv.best_params_

{'beta': 8.036046772187222}

In [ ]:
# searchCV on smooth data set
bscv_smooth = searchcv.BayesSearchCV(
    mdl, domain, n_iter=200, cv=cv, verbose=0, n_jobs=1, iid=True, n_points=3,
    error_score=-3e5)

def on_step(optim_result):
    score = bscv_smooth.best_score_
#     print("best score: %s" % score)

bscv_smooth.fit(data['smooth'].data, callback=on_step)

### GridSearchCV
As for the hyper-parameters tuning, one may choose to fix a grid of parameters and select the best ones.
For this we can use `GridSearchCV`, from the `scikit-learn` library.

In [ ]:
# data_grid = np.array(data.data).transpose(1,2,0)
param_grid=dict(alpha=np.logspace(-2,0,3), beta=np.logspace(-2,0,3),
                time_norm=[1, 2])

mdl = time_graph_lasso_.TimeGraphLassoForwardBackward(
    verbose=0, time_on_axis='last')
    
cv = ShuffleSplit(2, test_size=0.2)
ltgl = GridSearchCV(mdl, param_grid, cv=cv, verbose=1)
ltgl.fit(data_grid)